Again, we will use the high-level detect movement function, but instead of using the default values, you can use the optimal settings found in notebook 4 'parameterselection'. Besides viewing location detection results, it is possible to see the effects of the settings on raw movement detection. Also, we will show you how to run movement detection on a list of videofiles.

In [ ]:
import birdwatcher as bw
import birdwatcher.movementdetection as md
from birdwatcher.plotting import imshow_frame # birdwatcher has vizualization tools

import matplotlib.pyplot as plt
%matplotlib inline

### Select video fragment

In [ ]:
vfs = bw.VideoFileStream(r'..\videos\zf20s_low.mp4')

# optional: if you want to do movement detection only on part of the video
startat = '00:00:00'   # in HOURS:MM:SS
nframes = None

# specify h1, h2, w1, w2, or choose None to use the whole frame
roi = None   # region of interest
nroi = None   # nót region of interest

In [ ]:
# show roi and nroi in frame
if roi is not None:
    frame = vfs.iter_frames(startat=startat, nframes=1).draw_text(['roi'], org=(roi[0],roi[3]))
    imshow_frame(frame.peek_frame(), draw_rectangle=roi)

if nroi is not None:
    frame = vfs.iter_frames(startat=startat, nframes=1).draw_text(['nroi'], org=(nroi[0],nroi[3]))
    imshow_frame(frame.peek_frame(), draw_rectangle=nroi)

In [ ]:
# look at the chosen video fragment
vfs.iter_frames(startat=startat, nframes=nframes).show(framerate=150)

### Set parameters

First, decide which settings you would like, by adding one value in the list after each parameter. You could enter here the optimal settings you have found in the notebook 'parameterselection'. NOTE: that the values in this dictionary don't contain lists!

In [ ]:
settings = {'bgs_params':  {'History': 3,
                            'ComplexityReductionThreshold': 0.05,
                            'BackgroundRatio': 0.1,
                            'NMixtures': 7,
                            'VarInit': 15,
                            'VarMin': 10,
                            'VarMax': 75,
                            'VarThreshold': 70,
                            'VarThresholdGen': 9,
                            'DetectShadows': False,
                            'ShadowThreshold': 0.5,
                            'ShadowValue': 0},

            'processing':  {'color': False,   # booleans only
                            'resizebyfactor': 1,   # use '1' for no change in size
                            'blur': 10,   # use '0' for no blur
                            'morphologyex': True}}   # booleans only

To use a different background subtraction algorithm, just replace the parameters of the background subtractor with parameters of another algorithm (e.g. from BackgroundSubtractorKNN or BackgroundSubtractorLSBP).

### Run movement detection

In [ ]:
coords, coordscount, coordsmean = md.detect_movement(vfs, settings, startat, nframes, roi, nroi,
                                                     bgs_type=bw.BackgroundSubtractorMOG2,
                                                     analysispath='output/', ignore_firstnframes=50, 
                                                     overwrite=True, resultvideo=True)

In [ ]:
movementpath = f'output/movement_{vfs.filepath.stem}'
movementpath

The coordinate arrays are saved in the output/ directory within a 'movement' folder with the name of the videofilestream. Also a movementvideo (of the videofragment) is directly saved in the movement folder.

We can create a video of the coordinates results as well:

In [ ]:
coords.tovideo(f'{movementpath}/coordsvideo.mp4', framerate=vfs.avgframerate)

### Load coordinate arrays

In [ ]:
coords = bw.CoordinateArrays(f'{movementpath}/coords.darr')
coordscount = coords.get_coordcount()
coordsmean = coords.get_coordmean()

In [ ]:
coords.metadata

In [ ]:
coords.metadata['settings']

### Plot results

The coordscount shows the number of pixels that belong to the foreground, e.g. 'movement pixels', per frame. Higher peaks means more movement.

In [ ]:
plt.plot(coordscount)
plt.title('number of pixels above treshold')
plt.xlabel('framenumber')
plt.ylabel('number of pixels')

The coordsmean shows the mean coordinates per frame. This could be used to look at the location of the subject during the video. Note, there is a different graph to see the horizontal coordinates (left-rigth) and the vertical coordinates (top-bottom).

In [ ]:
plt.plot(coordsmean)
plt.title('coordinates of pixels above treshold')
plt.xlabel('frame number')
plt.ylabel('pixel coordinate')
plt.legend(['left-right', 'top-bottom'])

### Look at range of parameter values

If you still have some doubt about the optimal parameter values, or you just want to compare several settings by looking at the raw coordinates, you can run movement detection with several values for a specific parameter. For each setting a coordinate array and a movementvideo per value are directly saved. We also save a coordinate video.

In [ ]:
import copy
s = copy.deepcopy(settings)

In [ ]:
parameter = 'blur'
values = [0, 10]

for value in values:
    pathname = f'{movementpath}_{parameter}{value}'

    if parameter in s['bgs_params'].keys():
        s['bgs_params'][parameter] = value
    elif parameter in s['processing'].keys():
        s['processing'][parameter] = value

    coords, _, _ = md.detect_movement(vfs, s, startat, nframes, roi, nroi,
                                      bgs_type=bw.BackgroundSubtractorMOG2,
                                      analysispath=pathname, ignore_firstnframes=50, 
                                      overwrite=True, resultvideo=True)
    coords.tovideo(f'{pathname}/coordsvideo.mp4', framerate=vfs.avgframerate)

### Batch detect movement

Do movement detection on multiple videos, in which the same parameters values will be applied to each video. From each video a coordsarray and movementvideo will be saved.

In [ ]:
# to be implemented